In [ ]:
from analysis_processor import AnalysisProcessor

data_source = {
	"openai": "https://openai.com/policies/privacy-policy/",
	"anthropic": "https://www.anthropic.com/legal/privacy",
	# "perplexity": "https://www.perplexity.ai/hub/legal/privacy-policy",	# HTTPError: 403 Client Error: Forbidden for url: https://www.perplexity.ai/hub/legal/privacy-policy
	# "deepseek": "https://cdn.deepseek.com/policies/en-US/deepseek-privacy-policy.html", # Returns None
}
a = AnalysisProcessor(data_source, debug=False)
a.runAnalyses()

STARTING ALL policy processing
STARTING policy processing openai
STARTING policy processing anthropic
FINISHED chunk 14 in anthropic
total chunks for openai: 13
STARTING chunk 0 in openai
STARTING chunk 1 in openai
STARTING chunk 2 in openai
STARTING chunk 3 in openai
STARTING chunk 4 in openai
STARTING chunk 5 in openai
STARTING chunk 6 in openai
STARTING chunk 7 in openai
STARTING chunk 8 in openai
STARTING chunk 9 in openai
STARTING chunk 10 in openai
STARTING chunk 11 in openai
STARTING chunk 12 in openai
FINISHED chunk 5 in openai
total chunks for anthropic: 19
STARTING chunk 0 in anthropic
STARTING chunk 1 in anthropic
STARTING chunk 2 in anthropic
STARTING chunk 3 in anthropic
STARTING chunk 4 in anthropic
STARTING chunk 5 in anthropic
STARTING chunk 6 in anthropic
STARTING chunk 7 in anthropic
STARTING chunk 8 in anthropic
STARTING chunk 9 in anthropic
STARTING chunk 10 in anthropic
STARTING chunk 11 in anthropic
STARTING chunk 12 in anthropic
STARTING chunk 13 in anthropic
STA

In [ ]:
a.filterDuplicates()
# print("FINISHED FILTERING Retrieval Embeddings")


# print("FINISHED adding Retrieval Embeddings")
# self.saveData()

Strings to remove:
duplicate: Does the privacy policy affirm that the company does not knowingly use information from children under the age of 18?, main:Does the privacy policy affirm that the company does not knowingly collect information from children under the age of 18?
updating: {'8797960062c67485b46319247c3666b728326a5365ee8cf8f6be6cf6f19e295d': []}
with: {'8797960062c67485b46319247c3666b728326a5365ee8cf8f6be6cf6f19e295d': []}
duplicate: Does the privacy policy affirm that expressly consenting to the Privacy Policy confirms the user's consent to the processing of personal data?, main:Does the privacy policy affirm that expressly consenting to the Privacy Policy confirms the user's consent to the use of personal data?
updating: {'c00cfa5642b1a71eda79f3d57072784ecadf16a1176068b79cb240bfa5801d91': []}
with: {'c00cfa5642b1a71eda79f3d57072784ecadf16a1176068b79cb240bfa5801d91': []}
duplicate: Does the privacy policy affirm that personal data is used to communicate with the user?, main

ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': '* BatchEmbedContentsRequest.requests: requests must not be empty\n', 'status': 'INVALID_ARGUMENT'}}

In [ ]:
a.addFactEmbeddings()

NameError: name 'a' is not defined

In [ ]:
# a.runSubstringAnalysis("FACT_EMBEDDINGS")

arg_max: 0.8154822347987545
arg_max: 0.7785115614711727
arg_max: 0.7636897238462672
arg_max: 0.7876893859570298
arg_max: 0.8005100270620508
arg_max: 0.8160055460930307
arg_max: 0.8110761999104599
arg_max: 0.8124730738498347
arg_max: 0.7838888484205083
arg_max: 0.789587501634968
arg_max: 0.7884234204626059
arg_max: 0.7933264499111192
arg_max: 0.7713240494981017
arg_max: 0.7900870705308061
arg_max: 0.7900028959066683
arg_max: 0.7607020953557138
arg_max: 0.8132821762126792
arg_max: 0.8030040201322103
arg_max: 0.8348736555865537
arg_max: 0.8012844039902214
arg_max: 0.8142477179911228
arg_max: 0.7956246102158705
arg_max: 0.8090199758633856
arg_max: 0.7963495588556766
arg_max: 0.7864624050375516
arg_max: 0.7940792260262169
arg_max: 0.7960120570321789
arg_max: 0.794044418007121
arg_max: 0.7925871441055031
arg_max: 0.7905018934267262
arg_max: 0.7555205764885982
arg_max: 0.7345365341293062
arg_max: 0.7321064224731562
arg_max: 0.7605498868215159
arg_max: 0.7505203355061221
arg_max: 0.81052908736

In [3]:
a.runSubstringAnalysis("CROSS_FACTS")

ValueError: too many values to unpack (expected 2)

In [ ]:
cleanString = lambda x: re.sub(r"[^a-zA-Z]", "", re.sub(r"\\u[0-9a-fA-F]{4}", "", x))


def getMaxMatchSubstring(substring, superstring):
	clean_sub = cleanString(substring)

	low = 0
	high = len(clean_sub)
	best_match = None
	while low <= high:
		mid = (low + high) // 2
		current = clean_sub[:mid]

		if current in superstring:
			best_match = current
			low = mid + 1
		else:
			high = mid - 1

	return best_match


def getCleanedMatch(sub_match, substring, superstring):

	i = superstring.find(sub_match)
	if i == -1:
		return None

	target_alpha_len = len(cleanString(substring))

	if target_alpha_len == 0:
		return None

	low = 0
	high = len(superstring)
	end_index = high

	while low <= high:
		mid = (low + high) // 2

		current_alpha_len = len(cleanString(superstring[i:mid]))

		if current_alpha_len >= target_alpha_len:

			end_index = mid
			high = mid - 1
		else:
			low = mid + 1

	if high == 0:
		return None

	return superstring[i:end_index]


# [TODO]:
# - Supplemental RAG retreval
# 	- add alt embeddings for RETRIEVAL_DOCUMENT
# - Better question generation, (more general)
# - Clustering, filtering and pruning questions
# - Re-analysis mode
# - Make getCleanedMatch better for parts where we get multiple matches of substring edge case
# - Make extractContent more robust
# - Thinkabout where we can parellelise API calls


import datetime
import requests
from bs4 import BeautifulSoup
from markdownify import markdownify as md
import re
import datetime
import hashlib
import ast
from question_analysis import LLMInterface
from model_management import GeminiModel
import numpy as np
import json
import os
from google.genai import types
import concurrent.futures

QUESTIONS_FILE = "./questions_filter_after.json"
POLICIES_FILE = "./policies_testing.json"


def splitParargraphs(input_string):
	return input_string.split("\n\n")


def splitNewlines(input_string):
	while "\n\n" in input_string:
		input_string = input_string.replace("\n\n", "\n")
	return input_string.split("\n")


def __loadJson(filepath):
	if not os.path.exists(filepath):
		return {}
	try:
		with open(filepath, "r", encoding="utf-8") as f:
			return json.load(f)
	except json.JSONDecodeError:
		return {}


def __saveJson(filepath, data):
	with open(filepath, "w", encoding="utf-8") as f:
		json.dump(data, f, indent=4, default=str)


temp_cache_questions = __loadJson(QUESTIONS_FILE)


def _loadJson(filepath):
	if not os.path.exists(filepath):
		return {}
	try:
		with open(filepath, "r", encoding="utf-8") as f:
			return json.load(f)
	except json.JSONDecodeError:
		return {}


def _saveJson(filepath, data):
	with open(filepath, "w", encoding="utf-8") as f:
		json.dump(data, f, indent=4, default=str)


def loadQuestions(debug=True):
	if debug:
		return temp_cache_questions
	return _loadJson(QUESTIONS_FILE)


def loadPolicies():
	return _loadJson(POLICIES_FILE)


def updateQuestionEntry(question_text, new_policy_data):
	data = _loadJson(QUESTIONS_FILE)
	data[question_text].update(new_policy_data)
	_saveJson(QUESTIONS_FILE, data)


def updateQuestionEntry2(question_text, policy_hash, chunk_hash, new_policy_data):
	data = _loadJson(QUESTIONS_FILE)
	data[question_text][policy_hash][chunk_hash].update(new_policy_data)
	_saveJson(QUESTIONS_FILE, data)


def addNewQuestionChunk(question_text, policy_hash, chunk_hash, debug=True):
	if debug:
		temp_cache_questions[question_text]["policy_data"][policy_hash].update(
			{chunk_hash: []}
		)

	else:
		data = _loadJson(QUESTIONS_FILE)

		data[question_text]["policy_data"][policy_hash].update({chunk_hash: []})
		_saveJson(QUESTIONS_FILE, data)


def addNewQuestion(question_text, question_data, debug=True):
	if debug:
		temp_cache_questions[question_text] = question_data
	else:
		data = _loadJson(QUESTIONS_FILE)
		data[question_text] = question_data
		_saveJson(QUESTIONS_FILE, data)


def updateQuestionChunk(
	question_text, policy_hash, chunk_hash, new_substring_data, debug=True
):
	if new_substring_data == []:
		return
	if debug:
		temp_cache_questions[question_text]["policy_data"][policy_hash][chunk_hash].append(
			new_substring_data
		)
	else:
		data = _loadJson(QUESTIONS_FILE)
		data[question_text]["policy_data"][policy_hash][chunk_hash].append(new_substring_data)
		_saveJson(QUESTIONS_FILE, data)


def updateQuestionChunk(question_text, policy_hash, chunk_hash, new_substring_data):
	if new_substring_data == []:
		return
	data = _loadJson(QUESTIONS_FILE)
	data[question_text]["policy_data"][policy_hash][chunk_hash].append(new_substring_data)
	_saveJson(QUESTIONS_FILE, data)


def updatePolicyEntry(policy_name, new_version_data):
	data = _loadJson(POLICIES_FILE)
	data[policy_name].update(new_version_data)
	_saveJson(POLICIES_FILE, data)


def addNewPolicy(policy_name, policy_data):
	data = _loadJson(POLICIES_FILE)
	data[policy_name] = policy_data
	_saveJson(POLICIES_FILE, data)


def isInString(string, substring):
	return substring in string


def getSubstringIndices(string, substring):
	start_index = string.find(substring)
	end_index = start_index + len(substring)
	return (start_index, end_index)


import time


class AnalysisProcessor:

	modes = ["ANALYSIS", "DEBUG_QUESTION_GENERATION"]
	substring_modes = ["PROMPT", "FACT_EMBEDDINGS", "TESTING"]

	useRetreval = True
	MARKDOWN_LINK_PATTERN = re.compile(r"(\[.*?\])\((.*?)\)")
	URL_PLACEHOLDER = "(DYNAMIC_URL_REMOVED)"

	@staticmethod
	def extractContent(url, headers=None):
		if headers is None:
			headers = {
				"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
			}

			response = requests.get(url, headers=headers, timeout=10)
			response.raise_for_status()
			html_content = response.text

			return BeautifulSoup(html_content, "lxml").find("main")

	@staticmethod
	def extractMarkdown(main_content):
		return md(str(main_content), heading_style="ATX")

	@staticmethod
	def splitMarkdown(markdown_text):
		heading_pattern = r"^#{1,6}\s+.*"
		parts = re.split(heading_pattern, markdown_text, flags=re.MULTILINE)
		content_list = [part.strip() for part in parts[1:] if part.strip()]
		return content_list

	@staticmethod
	def removePreamble(markdown_text):
		pattern = r"\A.*?(?=^#\s)"
		cleaned_text = re.sub(pattern, "", markdown_text, flags=re.DOTALL | re.MULTILINE)
		return cleaned_text

	@staticmethod
	def normaliseMarkdownLinks(markdown_text):
		def replacer(match):
			return match.group(1) + AnalysisProcessor.URL_PLACEHOLDER

		normalized_text = AnalysisProcessor.MARKDOWN_LINK_PATTERN.sub(replacer, markdown_text)
		return normalized_text

	@staticmethod
	def getHash(data):
		return hashlib.sha256(data.encode("utf-8")).hexdigest()

	@staticmethod
	def processSimilarity(emb_a, emb_b):
		return np.dot(emb_a, emb_b) / (np.linalg.norm(emb_a) * np.linalg.norm(emb_b))

	@staticmethod
	def processUrl(policy_url):
		raw_policy_content = AnalysisProcessor.extractContent(policy_url)
		raw_markdown_content = AnalysisProcessor.extractMarkdown(raw_policy_content)
		policy_content = AnalysisProcessor.removePreamble(raw_markdown_content)
		normalised_content = AnalysisProcessor.normaliseMarkdownLinks(policy_content)
		policy_hash = AnalysisProcessor.getHash(normalised_content)
		return policy_content, policy_hash

	def getDuplicateQuestions(self, question_data):
		# saved_data = loadQuestions()
		preprocessed_questions = []
		for i, q in enumerate(question_data):
			embedding_vector = q["embedding_vector"]
			question = q["question"]
			if question in self.question_data:
				print(f"Exact duplicated: {question}")
				continue
			for k, v in self.question_data.items():
				trail_embedding = v["embedding_vector"]

				if AnalysisProcessor.processSimilarity(embedding_vector, trail_embedding) > 0.98:
					print(f"Near duplicate found: '{k}','{question}'")	# works suprisingly well

					preprocessed_questions.append([i, k])
					break

		return preprocessed_questions

	def isStored(self, policy_name, policy_hash):
		policy_data = loadPolicies()
		return (policy_name in policy_data) and (policy_hash in policy_data[policy_name])

	def swapSeenQuestions(self, with_embeddings):

		_ = list(with_embeddings.keys())[0]
		question_data = with_embeddings[_]
		preprocessed_questions = self.getDuplicateQuestions(question_data)

		modified_question_set = with_embeddings	# ast.literal_eval(_question_set)
		question_dict_key = list(modified_question_set.keys())[0]
		modified_question_data = modified_question_set[question_dict_key]
		for i in preprocessed_questions:
			modified_question_data[i[0]].update({"question": i[1]})

		return modified_question_data

	def processChunks(self, policy_content):
		proto_chunks = AnalysisProcessor.splitMarkdown(policy_content)
		_embeddings = self.analysis_model.client.models.embed_content(
			model="gemini-embedding-001",
			contents=proto_chunks,
			config=types.EmbedContentConfig(task_type="RETRIEVAL_DOCUMENT"),
		)
		embeddings = [e for e in _embeddings.embeddings]
		chunks = []
		for i, chunk in enumerate(proto_chunks):
			# if self.useRetreval and type(embeddings == list):
			chunks.append(
				{
					"chunk_content": {AnalysisProcessor.getHash(chunk): chunk},
					"retrieval_embedding_vector": embeddings[i].values,
				}
			)

		return chunks

	def __init__(self, data_source, mode=0, substring_mode=1):
		_mode = 0
		if type(mode) == int and mode < len(self.modes):
			_mode = mode
		_substring_mode = 1
		if type(substring_mode) == int and substring_mode < len(self.substring_modes):
			_substring_mode = substring_mode
		self.substring_mode = self.substring_modes[_substring_mode]
		self.mode = self.modes[_mode]
		self.data_source = data_source
		self.analysis_model = GeminiModel()
		self.model_interface = LLMInterface(self.analysis_model)
		self.loadData()

	def updatePolicyData(self, policy_content, policy_chunks, policy_name, policy_hash):
		fetch_date = datetime.datetime.now().__str__()

		policy_data = {
			policy_hash: {
				"policy_content": policy_content,
				"policy_chunks": policy_chunks,
				"fetch_date": fetch_date,
			}
		}
		if policy_name in loadPolicies():
			self.updatePolicyEntry(policy_name, policy_data)
		else:
			self.addNewPolicy(policy_name, policy_data)

	def addNewPolicy(self, policy_name, policy_data):
		self.policy_data[policy_name] = policy_data

	def loadData(self):
		self.question_data = loadQuestions(debug=False)
		self.policy_data = loadPolicies()

	def saveData(self):
		_saveJson(QUESTIONS_FILE, self.question_data)
		_saveJson(POLICIES_FILE, self.policy_data)

	def addNewQuestionChunk(self, question_text, policy_hash, chunk_hash, debug=True):

		self.question_data[question_text]["policy_data"][policy_hash].update({chunk_hash: []})

	def addNewQuestion(self, question_text, question_data, debug=True):

		self.question_data[question_text] = question_data

	def updateQuestionChunk(
		self, question_text, policy_hash, chunk_hash, new_substring_data
	):

		self.question_data[question_text]["policy_data"][policy_hash][chunk_hash].append(
			new_substring_data
		)

	def updatePolicyEntry(self, policy_name, new_version_data):
		# data = _loadJson(POLICIES_FILE)
		self.policy_data[policy_name].update(new_version_data)
		# _saveJson(POLICIES_FILE, data)

	def filterDuplicates(self):
		self.loadData()
		old_data = self.question_data.copy()
		# clean_data = {} # will be question, embedding_vector
		indexed_questions = []
		indexed_embeddings = []
		for k, v in old_data.items():
			indexed_questions.append(k)
			indexed_embeddings.append(v["embedding_vector"])
		array_indexed_embeddings = np.array(indexed_embeddings)
		array_indexed_questions = np.array(indexed_questions)

		sim_matrix = array_indexed_embeddings @ array_indexed_embeddings.T

		is_duplicate_matrix = np.triu(sim_matrix > 0.98, k=1)

		duplicates_mask = np.any(is_duplicate_matrix, axis=0)

		masters_all = np.argmax(is_duplicate_matrix, axis=0)

		indices_to_drop = np.where(duplicates_mask)[0]
		associated_masters = masters_all[duplicates_mask]
		strings_to_pop = array_indexed_questions[indices_to_drop]
		strings_overrides = array_indexed_questions[associated_masters]

		print(f"Strings to remove:")

		for i, key in enumerate(strings_to_pop):
			print(f"duplicate: {key}, main:{strings_overrides[i]}")
			policy_data = self.question_data[key]["policy_data"]
			for k, v in policy_data.items():
				if k in self.question_data[strings_overrides[i]]["policy_data"]:
					print(f"updating: {self.question_data[strings_overrides[i]]["policy_data"][k]}")
					print(f"with: {v}")
					self.question_data[strings_overrides[i]]["policy_data"][k].update(v)
				else:
					print(f"adding: {dict({k:v})}")
					print(f"to: {self.question_data[strings_overrides[i]]["policy_data"]}")

					self.question_data[strings_overrides[i]]["policy_data"].update({k: v})
			policy_data = self.question_data.pop(key)
		self.saveData()

	def processPolicyChunks(self, chunk, policy_hash, policy_name=None, index=0):
		print(f"STARTING chunk {index} in {policy_name}")
		chunk_hash = list(chunk["chunk_content"].keys())[0]
		chunk_content = list(chunk["chunk_content"].values())[0]

		_question_set: str = self.model_interface.generateQuestions(chunk_content)
		start_time = time.time()
		with_embeddings: dict = self.model_interface.processEmbeddings(
			_question_set, add_facts=False
		)
		reformed_question_set = list(with_embeddings.values())[0]
		for i in reformed_question_set:
			question = i["question"]
			if question in self.question_data:
				if policy_hash in self.question_data[question]["policy_data"]:
					if chunk_hash in self.question_data[question]["policy_data"][policy_hash]:
						continue
					else:
						self.addNewQuestionChunk(question, policy_hash, chunk_hash)

			else:
				self.addNewQuestion(
					question,
					{
						"embedding_vector": i["embedding_vector"],
						"policy_data": {policy_hash: {chunk_hash: []}},
					},
				)
		print(f"FINISHED chunk {index} in {policy_name}")

	def processPolicy(self, policy_name, policy_url):
		print(f"STARTING policy processing {policy_name}")

		policy_content, policy_hash = AnalysisProcessor.processUrl(policy_url)

		policy_chunks = self.processChunks(policy_content)
		self.updatePolicyData(policy_content, policy_chunks, policy_name, policy_hash)

		iterations = 0
		start_time = 0
		num_policy_chunks = len(policy_chunks)
		print(f"total chunks for {policy_name}: {num_policy_chunks}")
		executor = concurrent.futures.ThreadPoolExecutor(max_workers=num_policy_chunks)
		e_dict = dict()
		for index, chunk in enumerate(policy_chunks):
			e_dict[index] = executor.submit(
				self.processPolicyChunks, chunk, policy_hash, policy_name, index
			)
		executor.shutdown(wait=True)
		print(f"FINISHED ALL chunks for {policy_name}")

		# self.saveData()

	def addFactEmbeddings(self):
		question_emb = self.model_interface.processFactEmbeddings(self.question_data)
		for i in question_emb:
			self.question_data[i[0]].update({"retrieval_embedding_vector": i[1]})

	def runAnalyses(self, mode=None, limit_iterations=None):
		self.loadData()
		if mode is not None and mode in self.modes:
			self.mode = mode
		print("STARTING ALL policy processing")

		policies_total = len(list(self.data_source.keys()))
		executor = concurrent.futures.ThreadPoolExecutor(max_workers=policies_total)
		e = dict()
		for policy_name, policy_url in self.data_source.items():
			e[f"{[policy_name]}_{policy_url}"] = executor.submit(
				self.processPolicy, policy_name, policy_url
			)
		executor.shutdown(wait=True)
		print("FINISHED PROCESSING ALL Policies")
		self.saveData()

		self.filterDuplicates()
		print("FINISHED FILTERING Retrieval Embeddings")

		self.addFactEmbeddings()
		print("FINISHED adding Retrieval Embeddings")
		self.saveData()

	def getUpdates(self):
		self.loadData()
		needs_facts = []
		needs_prompts = []
		for k, v in self.question_data.items():
			policy_data = v["policy_data"]
			for l, w in policy_data.items():
				for m, x in w.items():
					if type(x) != list or len(x) == 0:
						needs_facts.append([k, l, m])
						needs_prompts.append([k, l, m])
					else:
						has_facts = False
						has_prompts = False
						for i in x:
							if has_facts and has_prompts:
								break
							if "method" not in i:
								needs_facts.append([k, l, m])
								needs_prompts.append([k, l, m])
								break
							else:
								method = i["method"]
								if method == "PROMPT":
									has_prompts = True
								elif method == "FACT_EMBEDDINGS":
									has_facts = True
						if not has_facts:
							needs_facts.append([k, l, m])
						if not has_prompts:
							needs_prompts.append([k, l, m])
		return needs_facts, needs_prompts

	def runSubstringAnalysis(self, substring_mode=None):
		self.loadData()

		if substring_mode is not None and substring_mode in self.substring_modes:
			self.substring_mode = substring_mode
		if self.substring_mode == "PROMPT":
			pass
		elif self.substring_mode == "FACT_EMBEDDINGS":
			# similarly use whole question retrieval_embedding_vector to identify any other missed duplicates
			needs_facts, _ = self.getUpdates()
			# policy_data = loadPolicies()
			# question_data = loadQuestions()

			for item in needs_facts:
				policy = None
				chunk = None
				for k, v in self.policy_data.items():
					if item[1] in v:
						policy = v[item[1]]
						break
				if policy is not None:
					chunks = policy["policy_chunks"]
					for c in chunks:
						chunk_data = c["chunk_content"]
						if item[2] in chunk_data:
							chunk = chunk_data[item[2]]
							break

					if chunk is not None:
						chunk_lines = splitNewlines(chunk)
						fact_embedding = self.question_data[item[0]]["retrieval_embedding_vector"]
						chunk_embeddings: list = [
							e.values
							for e in self.analysis_model.client.models.embed_content(
								model="gemini-embedding-001",
								contents=chunk_lines,
								config=types.EmbedContentConfig(task_type="RETRIEVAL_DOCUMENT"),
							).embeddings
						]
						mat = np.dot(chunk_embeddings, fact_embedding)
						idx = np.argmax(mat)
						print(mat[idx])
						substring = chunk_lines[idx]
						substring_indices = getSubstringIndices(policy["policy_content"], substring)
						question_update = {
							"substring": substring,
							"substring_indices": substring_indices,
							"method": "FACT_EMBEDDINGS",
						}
						self.updateQuestionChunk(*item, question_update)
				self.saveData()

		elif self.substring_mode == "TESTING":
			needs_facts, needs_prompts = AnalysisProcessor.getUpdates()
			print(len(needs_facts), len(needs_prompts))

In [ ]:
data_source = {
	"openai": "https://openai.com/policies/privacy-policy/",
	# "anthropic": "https://www.anthropic.com/legal/privacy",
	# "perplexity": "https://www.perplexity.ai/hub/legal/privacy-policy",	# HTTPError: 403 Client Error: Forbidden for url: https://www.perplexity.ai/hub/legal/privacy-policy
	# "deepseek": "https://cdn.deepseek.com/policies/en-US/deepseek-privacy-policy.html", # Returns None
}

In [ ]:
print(len(list(temp_cache_questions.keys())))
temp_cache_questions.keys()

In [ ]:
a = AnalysisProcessor(data_source)
a.runAnalyses()

In [ ]:
a = AnalysisProcessor(data_source, mode=1)

a.runSubstringAnalysis("FACT_EMBEDDINGS")

In [ ]:
import numpy as np

# 1. Setup: Create dummy data for demonstration
# In this scenario:
# Index 2 is a duplicate of Index 0
# Index 3 is a duplicate of Index 1
# Index 4 is a duplicate of Index 0 (and Index 2)
embeddings = np.array(
	[
		[1.0, 0.0],	# 0: Original A
		[0.0, 1.0],	# 1: Original B
		[1.0, 0.0],	# 2: Copy of A
		[0.0, 1.0],	# 3: Copy of B
		[1.0, 0.0],	# 4: Copy of A
	]
)

# 2. Normalize (Standard procedure for cosine similarity via dot product)
norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
normalized_emb = embeddings / norms

# 3. Compute Similarity Matrix
sim_matrix = normalized_emb @ normalized_emb.T

is_duplicate_matrix = np.triu(sim_matrix > 0.98, k=1)

duplicates_mask = np.any(is_duplicate_matrix, axis=0)

masters_all = np.argmax(is_duplicate_matrix, axis=0)

indices_to_drop = np.where(duplicates_mask)[0]
associated_masters = masters_all[duplicates_mask]

# Output
print(f"Indices to drop:    {indices_to_drop}")
print(f"Associated Masters: {associated_masters}")

# Verification Loop
for drop, master in zip(indices_to_drop, associated_masters):
	print(f"Item {drop} is a duplicate of Item {master}")

In [ ]:
qs = loadQuestions(debug=False)
for k in qs:
	print(k)

In [ ]:
a.runSubstringAnalysis("TESTING")

In [ ]:
question_data = temp_cache_questions.copy()
import numpy as np
from scipy.spatial.distance import pdist, squareform


def get_most_similar_mahalanobis(question_data):
	"""
	Identifies the two most semantically similar strings using Mahalanobis distance.

	Args:
	    question_data (dict): Dictionary of form {string: {"embedding_vector": [list]}}

	Returns:
	    tuple: (string_A, string_B, distance_score)
	"""

	keys = list(question_data.keys())

	if len(keys) < 2:
		return None, None, 0.0

	X = np.array([question_data[k]["embedding_vector"] for k in keys])

	cov_matrix = np.cov(X, rowvar=False)

	inv_cov_matrix = np.linalg.pinv(cov_matrix)

	condensed_distances = pdist(X, metric="mahalanobis", VI=inv_cov_matrix)

	min_index_condensed = np.argmin(condensed_distances)
	min_distance = condensed_distances[min_index_condensed]

	dist_matrix = squareform(condensed_distances)

	np.fill_diagonal(dist_matrix, np.inf)

	i, j = np.unravel_index(np.argmin(dist_matrix), dist_matrix.shape)

	return keys[i], keys[j], min_distance


get_most_similar_mahalanobis(question_data)